In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
import json

In [2]:
inq = pd.read_csv('/content/inquiries.csv')

In [3]:
inq.head()

,id,status,type,loan_amount,is_buying_business,who_is_buying_business,first_name,last_name,is_primarily_selling_or_providing_services,is_operating_for_profit,...,is_for_profit_subsidiary_of_nonprofit,is_pawn_shop,is_owned_or_controlled_by_native_american_tribe,user_id,organization_id,last_borrower_edit_at,created_at,updated_at,submitted_at,archived_at
0,db7a946b-8d2e-4575-9e78-49c6d4a744d4,awaiting help,{export},71766400,f,NaN,Samantha,Rosenbaum,t,f,...,f,f,f,k0lrpzsj2ua337dymsmbg,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-10-15 21:23:07.976+00,2025-10-07 21:23:07.976+00,NaN,NaN,NaN
1,f5003457-171c-4472-943b-f409d4d51ac8,ineligible,{workingCapital},8323342,f,NaN,Fiona,Wilderman,t,f,...,f,f,f,szn372si8psxs97z5fne3,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-10-21 05:07:21.517+00,2025-10-16 05:07:21.517+00,NaN,NaN,NaN
2,8395b9c8-0672-462e-a12d-4ae062b9e619,awaiting help,{workingCapital},88975659,f,NaN,Alene,Williamson,f,f,...,f,f,f,m8g55bryzqfuhjgxc4oba,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-09-29 08:14:04.693+00,2025-09-16 08:14:04.693+00,NaN,NaN,NaN
3,3236dfe7-790c-408d-868e-4d36f3923a5e,forwarded,{acquisition},72706313,t,me,Cordell,Sporer,t,t,...,f,f,f,fk1kt1qy12q05hrq2mpib,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-10-18 20:36:08.209+00,2025-10-13 20:36:08.209+00,NaN,NaN,NaN
4,fcc58719-edd3-4ed6-a680-e8018b8a2c03,archived,{commercialRealEstate},56285347,f,NaN,Mya,Krajcik,t,t,...,f,f,f,hmldmt9ggrghxatwg35a1,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-09-30 02:01:56.686+00,2025-09-22 02:01:56.686+00,NaN,NaN,NaN


In [4]:
inq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 34 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   id                                                      612 non-null    object 
 1   status                                                  612 non-null    object 
 2   type                                                    612 non-null    object 
 3   loan_amount                                             612 non-null    int64  
 4   is_buying_business                                      612 non-null    object 
 5   who_is_buying_business                                  157 non-null    object 
 6   first_name                                              612 non-null    object 
 7   last_name                                               612 non-null    object 
 8   is_primarily_selling_or_providing_servic

In [5]:
inq = inq.drop(columns=["archived_at", "submitted_at","updated_at","created_at","last_borrower_edit_at","first_name","last_name","who_is_buying_business","user_id","organization_id"])

In [6]:
inq.head()

,id,status,type,loan_amount,is_buying_business,is_primarily_selling_or_providing_services,is_operating_for_profit,is_primarily_lending_business,is_pyramid_sale_distribution_plan,is_deriving_gambling_revenue,...,is_associate_with_criminal_history,is_equity_owned_by_lender_or_cdc,is_presenting_prurient_performances,is_defaulted_on_federal_loan,is_engaged_in_political_or_lobbying_activities,is_speculative_in_nature,is_primarily_earning_from_rent_interest_or_investments,is_for_profit_subsidiary_of_nonprofit,is_pawn_shop,is_owned_or_controlled_by_native_american_tribe
0,db7a946b-8d2e-4575-9e78-49c6d4a744d4,awaiting help,{export},71766400,f,t,f,f,f,f,...,f,f,t,f,f,f,f,f,f,f
1,f5003457-171c-4472-943b-f409d4d51ac8,ineligible,{workingCapital},8323342,f,t,f,f,f,f,...,f,f,f,f,f,f,f,f,f,f
2,8395b9c8-0672-462e-a12d-4ae062b9e619,awaiting help,{workingCapital},88975659,f,f,f,f,f,f,...,f,f,f,f,f,f,f,f,f,f
3,3236dfe7-790c-408d-868e-4d36f3923a5e,forwarded,{acquisition},72706313,t,t,t,f,f,f,...,f,f,t,f,f,f,f,f,f,f
4,fcc58719-edd3-4ed6-a680-e8018b8a2c03,archived,{commercialRealEstate},56285347,f,t,t,f,f,f,...,f,f,f,f,f,f,f,f,f,f


In [7]:
inq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 24 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   id                                                      612 non-null    object
 1   status                                                  612 non-null    object
 2   type                                                    612 non-null    object
 3   loan_amount                                             612 non-null    int64 
 4   is_buying_business                                      612 non-null    object
 5   is_primarily_selling_or_providing_services              612 non-null    object
 6   is_operating_for_profit                                 612 non-null    object
 7   is_primarily_lending_business                           612 non-null    object
 8   is_pyramid_sale_distribution_plan                 

In [8]:
inq = inq.replace(0, pd.NA).dropna(how="any")

In [9]:
inq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 24 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   id                                                      612 non-null    object
 1   status                                                  612 non-null    object
 2   type                                                    612 non-null    object
 3   loan_amount                                             612 non-null    int64 
 4   is_buying_business                                      612 non-null    object
 5   is_primarily_selling_or_providing_services              612 non-null    object
 6   is_operating_for_profit                                 612 non-null    object
 7   is_primarily_lending_business                           612 non-null    object
 8   is_pyramid_sale_distribution_plan                 

In [10]:
cols = ["is_buying_business", "is_primarily_selling_or_providing_services", "is_operating_for_profit","is_primarily_lending_business","is_pyramid_sale_distribution_plan","is_deriving_gambling_revenue","is_engaged_in_illegal_activity","is_private_club_with_limited_membership","is_government_owned_entity","is_loan_packager","is_associate_with_criminal_history","is_equity_owned_by_lender_or_cdc","is_presenting_prurient_performances","is_defaulted_on_federal_loan","is_engaged_in_political_or_lobbying_activities","is_speculative_in_nature","is_primarily_earning_from_rent_interest_or_investments","is_for_profit_subsidiary_of_nonprofit","is_pawn_shop","is_owned_or_controlled_by_native_american_tribe"]
inq[cols] = inq[cols].apply(
    lambda s: s.astype("string").str.strip().str.lower().eq("t").astype("Int8")
)

In [11]:
inq.head()

,id,status,type,loan_amount,is_buying_business,is_primarily_selling_or_providing_services,is_operating_for_profit,is_primarily_lending_business,is_pyramid_sale_distribution_plan,is_deriving_gambling_revenue,...,is_associate_with_criminal_history,is_equity_owned_by_lender_or_cdc,is_presenting_prurient_performances,is_defaulted_on_federal_loan,is_engaged_in_political_or_lobbying_activities,is_speculative_in_nature,is_primarily_earning_from_rent_interest_or_investments,is_for_profit_subsidiary_of_nonprofit,is_pawn_shop,is_owned_or_controlled_by_native_american_tribe
0,db7a946b-8d2e-4575-9e78-49c6d4a744d4,awaiting help,{export},71766400,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,f5003457-171c-4472-943b-f409d4d51ac8,ineligible,{workingCapital},8323342,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8395b9c8-0672-462e-a12d-4ae062b9e619,awaiting help,{workingCapital},88975659,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3236dfe7-790c-408d-868e-4d36f3923a5e,forwarded,{acquisition},72706313,1,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,fcc58719-edd3-4ed6-a680-e8018b8a2c03,archived,{commercialRealEstate},56285347,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
def convert_type_to_list(type_string):
    # Check if it's a string (handles potential NaN values)
    if isinstance(type_string, str):
        # Remove the curly braces {}
        stripped_string = type_string.strip('{}')
        # If the stripped string is empty (was "{}"), return an empty list
        if not stripped_string:
            return []
        # Split the string by comma
        return stripped_string.split(',')
    else:
        # Return an empty list for NaN or other non-string values
        return []

# Apply the function to the 'type' column
inq['type'] = inq['type'].apply(convert_type_to_list)

In [15]:
inq.head(20)

,id,status,type,loan_amount,is_buying_business,is_primarily_selling_or_providing_services,is_operating_for_profit,is_primarily_lending_business,is_pyramid_sale_distribution_plan,is_deriving_gambling_revenue,...,is_associate_with_criminal_history,is_equity_owned_by_lender_or_cdc,is_presenting_prurient_performances,is_defaulted_on_federal_loan,is_engaged_in_political_or_lobbying_activities,is_speculative_in_nature,is_primarily_earning_from_rent_interest_or_investments,is_for_profit_subsidiary_of_nonprofit,is_pawn_shop,is_owned_or_controlled_by_native_american_tribe
0,db7a946b-8d2e-4575-9e78-49c6d4a744d4,awaiting help,[export],71766400,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,f5003457-171c-4472-943b-f409d4d51ac8,ineligible,[workingCapital],8323342,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8395b9c8-0672-462e-a12d-4ae062b9e619,awaiting help,[workingCapital],88975659,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3236dfe7-790c-408d-868e-4d36f3923a5e,forwarded,[acquisition],72706313,1,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,fcc58719-edd3-4ed6-a680-e8018b8a2c03,archived,[commercialRealEstate],56285347,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,64aa7167-3368-47a1-9d2c-709e510eee6c,archived,[acquisition],55718351,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,496b3af8-527e-4403-95b0-919fafc3cff9,ineligible,[equipmentPurchase],1672482,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,de867cf5-0e27-4932-8cc8-28a01bcd652b,awaiting help,[commercialRealEstate],54616924,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,e34fcfd5-6b6b-4f66-8d0b-67653b68fffe,awaiting info,[workingCapital],62065372,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,598e1211-fe95-4cf4-9f8b-ef909928f13c,ineligible,"[acquisition, workingCapital, supplies, commer...",74874172,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
inq.to_csv("inq.csv", index=False)